In [1]:
import flask
import tensorflow as tf
import cv2
import numpy as np
import skimage
import os
from skimage import io
import matplotlib.pyplot as plt
import random
from flask import Flask


/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def predict():
    #file_name = flask.request.args['/Users/tunchiehhsu/Desktop/googliser/googliser/smashed_boxes/google-image(0007).jpeg']
    file_name = io.imread('/Users/tunchiehhsu/Desktop/googliser/googliser/smashed_boxes/google-image(0007).jpeg',as_gray = True)
    reshape_img = cv2.resize(file_name, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
    input_img = reshape_img.flatten()
    raw = input_img
    #raw = sess.run(model,feed_dict={x:input_img})
    return sendResponse({nameOfTheCharacter: raw})

In [3]:
def init():
    global x,predict_tensor,data,sess
    tf.reset_default_graph() # why?
    initt = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(initt)

    new_saver = tf.train.import_meta_graph('./checkpoint_dir/MyModel.meta')
    new_saver.restore(sess, tf.train.latest_checkpoint('./checkpoint_dir'))

    graph = tf.get_default_graph()
    x = graph.get_operation_by_name('xs').outputs[0]
    predict_tensor = tf.get_collection("pred_network")[0]

    ### read data
    filepaths = []
    folder = "/Users/tunchiehhsu/Desktop/googliser/googliser/damaged_packages"
    for file in os.listdir(folder):
            if file != ".DS_Store":
                filepath = os.path.join(folder, file)
                filepaths.append(filepath)
    data = []
    for path in filepaths:
        # gray scale
        img = io.imread(path,as_gray = True)
        # reshape to uniform size
        reshape_img = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
        data.append(reshape_img.flatten())

    data = np.array(data)

In [ ]:
app = Flask(__name__)
# Cross origin support
def sendResponse(responseObj):
    response = flask.jsonify(responseObj)
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Methods', 'GET')
    response.headers.add('Access-Control-Allow-Headers', 'accept,content-type,Origin,X-Requested-With,Content-Type,access_token,Accept,Authorization,source')
    response.headers.add('Access-Control-Allow-Credentials', True)
    return response

@app.route('/')
def classify():
    results = sess.run(predict_tensor,{x: data})
    
    return sendResponse({'prediction': results.tolist()})


if __name__ == '__main__':
    init()
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

INFO:tensorflow:Restoring parameters from ./checkpoint_dir/MyModel


 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Sep/2018 00:19:26] "GET / HTTP/1.1" 200 -
